<a href="https://colab.research.google.com/github/WillemAdriaens3D/FullControl-experiments/blob/main/models/colab/Chladni%20Figures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FullControl design template

*<<< check out demo models [here](https://github.com/FullControlXYZ/fullcontrol/tree/master/models/README.md) >>>*
  
press ctrl+F9 to run all cells in this notebook, or press shift+enter to run each cell sequentially

if you change one of the code cells, make sure you run it and all subsequent cells again (in order)

*this document is a jupyter notebook - if they're new to you, check out how they work: [link](https://www.google.com/search?q=ipynb+tutorial), [link](https://jupyter.org/try-jupyter/retro/notebooks/?path=notebooks/Intro.ipynb), [link](https://colab.research.google.com/)*
### be patient :)

the next code cell may take a while because running it causes several things to happen:
- connect to a google colab server -> download the fullcontrol code -> install the fullcontrol code

check out [other tutorials](https://github.com/FullControlXYZ/fullcontrol/blob/master/tutorials/README.md) to understand the python code for the FullControl design

In [5]:
if 'google.colab' in str(get_ipython()):
  !pip install git+https://github.com/FullControlXYZ/fullcontrol --quiet
import fullcontrol as fc
from google.colab import files

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [31]:
# printer/gcode parameters

design_name = 'my_design'
nozzle_temp = 210
bed_temp = 60
print_speed = 1000
fan_percent = 100
printer_name='generic' # generic / ultimaker2plus / prusa_i3 / ender_3 / cr_10 / bambulab_x1 / toolchanger_T0

In [48]:
# design parameters

EW = 0.8 # extrusion width
EH = 0.4 # extrusion height (and layer height)
initial_z = EH*0.4 # initial nozzle position is set to 0.6x the extrusion height to get a bit of 'squish' for good bed adhesion
layers = 200

In [47]:
# generate the design (make sure you've run the above cells before running this cell)

import math
from math import cos, tau, sin

# Parameters
start_radius = 75    # Beginradius (mm)
end_radius = 20     # Eindradius (mm)
height = 200         # Hoogte over de Z-as (mm)
resolution = 200    # Aantal stappen voor het patroon
kx = 2              # Chladni-modus in de x-richting
ky = 3              # Chladni-modus in de y-richting
amplitude = 3       # Amplitude van het patroon (intensiteit)
segments = 360     # Aantal zijden van de veelhoek (360 voor een cirkel)
curvature = 2       # Curvature parameter (1 for linear, >1 for concave, <1 for convex)

# Helperfunctie: Chladni-patroon
def chladni_pattern(r, theta, z):
    # Omgerekend naar cartesiaanse coördinaten
    x = r * math.cos(theta)
    y = r * math.sin(theta)
    return math.sin(kx * x) * math.sin(ky * y)

# Generate Chladni-pattern over a spiral shape with curvature
points = []
for i in range(resolution):
    z = i / resolution * height
    # Apply curvature to the radius calculation
    radius = start_radius + (end_radius - start_radius) * ((z / height)**curvature)
    for j in range(segments):
        theta = 2 * math.pi * j / segments
        displacement = amplitude * chladni_pattern(radius, theta, z)
        r = radius + displacement
        x = r * math.cos(theta)
        y = r * math.sin(theta)
        points.append((x, y, z))

# Output points for FullControl XYZ
for point in points:
    print(f"{point[0]},{point[1]},{point[2]}")

# ... (Rest of your code remains the same) ...

Streaminguitvoer ingekort tot de laatste 5000 regels.
19.971358433291513,16.75795949548417,186.0
21.365762089698713,18.572973627911793,186.0
20.227179988959204,18.212634666799204,186.0
19.464989944447918,18.15139677471557,186.0
20.95503439445609,20.236041490422394,186.0
21.275970997917106,21.275970997917103,186.0
19.449709699054242,20.14076398779712,186.0
18.979344848446857,20.35285555224256,186.0
20.023232351167316,22.238052436539178,186.0
19.290074407356865,22.190692171165875,186.0
17.374600363474325,20.706242403059193,186.0
17.31759434985399,21.38544802066216,186.0
18.0629700003953,23.119547304560577,186.0
16.958992855205047,22.505343648397382,186.0
15.386226760092727,21.177324336861393,186.0
15.64648158910355,22.345491494005422,186.0
16.21768470852284,24.04370634850188,186.0
15.10206105899726,23.25513470613843,186.0
13.827558202457508,22.128718843717554,186.0
14.156525654865886,23.56041518876201,186.0
14.652326096880236,25.37857324886394,186.0
13.702215171821942,24.719450522971417,

In [49]:
import fullcontrol as fc
import math

# ... (Your existing design generation code) ...

# Create a list of fc.Point objects from the generated points
fc_points = [fc.Point(x=p[0], y=p[1], z=p[2]) for p in points]

# Preview the design using fc.transform and fc.PlotControls, pass points directly
fc.transform(fc_points, 'plot', fc.PlotControls(style='line', zoom=0.7))

In [50]:
# generate and save gcode

gcode_controls = fc.GcodeControls(
    printer_name=printer_name,

    initialization_data={
        'primer': 'front_lines_then_y',
        'print_speed': print_speed,
        'nozzle_temp': nozzle_temp,
        'bed_temp': bed_temp,
        'fan_percent': fan_percent,
        'extrusion_width': EW,
        'extrusion_height': EH})
# Replace 'steps' with 'fc_points' to use the generated Chladni pattern points
gcode = fc.transform(fc_points, 'gcode', gcode_controls)
open(f'{design_name}.gcode', 'w').write(gcode)
files.download(f'{design_name}.gcode')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### please tell us what you're doing with FullControl!

- tag FullControlXYZ on social media ([twitter](https://twitter.com/FullControlXYZ), [instagram](https://www.instagram.com/fullcontrolxyz/), [linkedin](https://www.linkedin.com/in/andrew-gleadall-068587119/), [tiktok](https://www.tiktok.com/@fullcontrolxyz))
- email [info@fullcontrol.xyz](mailto:info@fullcontrol.xyz)
- post on the [subreddit](https://reddit.com/r/fullcontrol)
- post in the [github discussions or issues tabs](https://github.com/FullControlXYZ/fullcontrol/issues)

in publications, please cite the original FullControl paper and the github repo for the new python version:

- Gleadall, A. (2021). FullControl GCode Designer: open-source software for unconstrained design in additive manufacturing. Additive Manufacturing, 46, 102109.
- Gleadall, A. and Leas, D. (2023). FullControl [electronic resource: python source code]. available at: https://github.com/FullControlXYZ/fullcontrol